In [ ]:
!pip install --upgrade jax jaxlib

In [ ]:
import jax  # NOTE: jax-0.7.2, jaxlib-0.7.2 the lib is constantl developing so it might be outdated
import jax.numpy as jnp
import numpy as np

from jax import grad, jit, vmap, pmap

from jax import random
import matplotlib.pyplot as plt
from copy import deepcopy
from typing import Tuple, NamedTuple
import functools

In [ ]:
g = 0.

def impure_uses_global(x):
    return x + g

print(f"First call: {jit(impure_uses_global)(4.)}")
g = 10.  # update global state


print(f"Second call: {jit(impure_uses_global)(5.)}")  # used cached value

In [ ]:
seed = 0
state = jax.random.PRNGKey(seed)

state1, state2 = jax.random.split(state)

In [ ]:
class Counter:
    def __init__(self):
        self.n = 0

    def count(self) -> int:
        self.n += 1
        return self.n

    def reset(self):
        self.n = 0


counter = Counter()

for _ in range(3):
    print(counter.count())

In [ ]:
counter.reset()
fast_count = jax.jit(counter.count)

for _ in range(3):  # oops, it's not working as it's supposed to be
    print(fast_count())

In [ ]:
from jax import make_jaxpr


counter.reset()
print(make_jaxpr(counter.count)())

In [ ]:
CounterState = int

class CounterV2:
    def count(self, n: CounterState) -> Tuple[int, CounterState]:
        return n + 1, n + 1

    def reset(self) -> CounterState:
        return 0


counter = CounterV2()
state = counter.reset()

for _ in range(3):
    value, state = counter.count(state)
    print(value)

In [ ]:
state = counter.reset()
fast_count = jax.jit(counter.count)

for _ in range(3):
    value, state = counter.count(state)
    print(value)

In summary we used the following rule to convert a stateful class:
```python
class StatefulClass

    state: State

    def stateful_method(*args, **kwargs) -> Output:
        NotImplemented

```
into a class of the form:
```python
class StatelessClass

    def stateless_method(state: State, *args, **kwargs) -> (Output, State):
        NotImplemented

```
Nice - we figured an equivalent way to handle states without introducing the side-effects.

This brings us 1 step closer to building neural networks! 🥳

We still need to find a way to handle gradients when dealing with big NNs.

In nutshell, jax requires random_key/state everywhere

## Enter PyTree 🌳
Before we start - why are gradients a problem in the first place?

### PyTree basics

In [ ]:
f = lambda x, y, z, w: x**2 + y**2 + z**2 + w**2

# JAX: .backward() is not that great
# also JAX:
x, y, z, w = [1. for _ in range(4)]
dfdx, dfdy, dfdz, dfdw = grad(f, argnums=(0, 1, 2, 3))(x, y, z, w)
print(dfdx, dfdy, dfdz, dfdw)

# Great now we just need to update our params!
# lr = 0.001
# x -= lr*dfdx
# y -= lr*dfdy
# ... (175B lines later)
# w -= lr*dfdw

# No, no, no.
# We do have a better way.

No, no, no. We do have a better way!

We want to, more naturally, wrap our params in some more complex data structures like dictionaries, etc.

JAX knows how to deal with these! The answer is called a PyTree.

In [ ]:
# A contrived example for pedagogical purposes
# (if your mind needs to attach some semantics to parse this - treat it as model params)

pytree_example = [
    [1, 'a', object()],
    (1, (2, 3), ()),
    [1, {'k': 2, 'k2': (3, 4)}, 5],
    {'a': 2, 'b': (2, 3)},
    jnp.array([1, 2, 3]),
    jnp.array([[1, 2, 3]]),
]

for pytree in pytree_example:
    leaves = jax.tree.leaves(pytree)
    print(f"{repr(pytree):<45} has {len(leaves)} leaves: {leaves}")

In [ ]:
# How do we manipulate PyTrees?

list_of_list = [
    {'a': 3},
    [1, 2, 3],
    [1, 2],
    [1, 2, 3, 4]
]

# For single arg functions use tree_map
# tree_map iterates through leaves and applies the lambda function
# NOTE jax.tree_util.tree_map is an alias of jax.tree.map().
print(jax.tree_util.tree_map(lambda x: x*2, list_of_list))

In [ ]:
another_list_of_lists = list_of_list
# jax.tree_util.tree_multimap, replace it with jax.tree.map as they are functionally identical in this context.
print(jax.tree.map(lambda x, y: x + y, list_of_list, another_list_of_lists))

In [ ]:
# PyTrees need to have the same structure if we are to apply tree.map()
another_list_of_lists = deepcopy(list_of_list)
another_list_of_lists.append([23])  # not the same structure anymore
print(jax.tree.map(lambda x, y: x + y, list_of_list, another_list_of_lists))  # ValueError

### Less contrived example: training a toy MLP (multi-layer perceptron) model

In [ ]:
def init_mlp_params(layer_width):
    params = []

    # Allocate weights and biases (model parameters)
    # Notice: we're not using JAX's PRNG here - doesn't matter for this simple example
    for n_in, n_out in zip(layer_width[:-1], layer_width[1:]):
        params.append(
            dict(
                weights=np.random.normal(size=(n_in, n_out)) * np.sqrt(2. / n_in),
                bias=np.ones(shape=(n_out,))
            )
        )
    return params

# Instantiate a single input - single output, 3 layer (2 hidden layers) deep MLP
params = init_mlp_params([1, 128, 128, 1])

# Another example of how we might use tree_map - verify that shapes make sense:
jax.tree.map(lambda x: x.shape, params)

In [ ]:
def forward(params, x):
    *hidden, last = params

    for layer in hidden:
        x = jax.nn.relu(jnp.dot(x, layer["weights"]) + layer["bias"])

    return jnp.dot(x, last["weights"]) + last["bias"]


def loss_fn(params, x, y):
    return jnp.mean((forward(params, x) - y) ** 2)  # MSE Loss


lr = 0.0001

@jit # notice how we do jit only at the highest level - XLA will have plenty of space to optimize
def update(params, x, y):

    # Note that grads is a pytree with the same structure as params.
    # grad is one of the many JAX functions that has built-in support for pytrees!
    grads = jax.grad(loss_fn)(params, x, y)

    # Task: analyze grads and make sure it has the same structure as param

    # SGD update
    return jax.tree.map(lambda p, g: p - lr * g, params, grads)

In [ ]:
xs = np.random.normal(size=(128, 1))
ys = xs**2  # let's learn how to regress a parabola

# Task experiment a bit with other functions (polynomials, sin, etc.)

num_epochs = 5000
for _ in range(num_epochs):
    params = update(params, xs, ys)  # again our lovely pattern

plt.scatter(xs, ys, label="GT", c="g", marker=".")
plt.scatter(xs, forward(params, xs), label="Model prediction", c="r", marker=".")
plt.legend()
plt.show()

Wohoo! We've trained our first neural network! 🥳🥳🥳

In order to be able to build NN libs and layers such as nn.Linear (PyTorch syntax), etc. we need a couple more tricks up our sleeves! 🔥

### Custom PyTrees ⚙️

In [ ]:
class MyContainer:  # this could be a linear layer a conv layer or whatever
    def __init__(self, name: str, a: int, b: int, c: int):
        self.name = name
        self.a = a
        self.b = b
        self.c = c

In [ ]:
example_pytree = [MyContainer("Alice", 1, 2, 3), MyContainer("Bob", 1, 2, 3)]  # 8 leaves? Right? Noup.

leaves = jax.tree.leaves(example_pytree)
print(f"{repr(example_pytree):<45}\n has {len(leaves)} leaves:\n {leaves}")

In [ ]:
print(jax.tree.map(lambda x: x + 1, example_pytree))  # this will not work :/ it'd be nice if it did

In [ ]:
# Let's get it to work! We'll need to define 2 functions (flatten/unflatten)

def flatten_MyContainer(container):
    """Returns an iterable over container contents, and aux data."""
    flatten_contents = [container.a, container.b, container.c]

    # we don't want the name to appear as a child, so it is auxiliary data.
    # auxiliary data is usually a description of the structure of a node,
    # e.g., the keys of a dict -- anything that isn't a node's children.
    aux_data = container.name
    return flatten_contents, aux_data


def unflatten_MyContainer(aux_data, flat_contents):
    """Converts aux data and the flat contents into a MyContainer."""
    return MyContainer(aux_data, *flat_contents)

# Register a custom PyTree node
jax.tree_util.register_pytree_node(MyContainer, flatten_MyContainer, unflatten_MyContainer)

In [ ]:
# Let's try again!
leaves = jax.tree.leaves(example_pytree)
print(f"{repr(example_pytree):<45}\n has {len(leaves)} leaves:\n {leaves}")

In [ ]:
result = jax.tree.map(lambda x: x + 1, example_pytree)
print(jax.tree.leaves(result))  # it works now as expected!

In [ ]:
# Finally a common gotcha working with PyTrees: mistaking nodes for leaves/children

zeros_tree = [jnp.zeros((2, 3)), jnp.zeros((3, 4))]
print(zeros_tree)

# Try to make another tree with ones instead of zeros
shapes = jax.tree.map(lambda x: x.shape, zeros_tree)
print(shapes)

ones_tree = jax.tree.map(jnp.ones, shapes)
print(ones_tree)

In [ ]:
# Finally a common gotcha working with PyTrees: mistaking nodes for leaves/children

zeros_tree = [jnp.zeros((2, 3)), jnp.zeros((3, 4))]
print(zeros_tree)

# Try to make another tree with ones instead of zeros
ones_tree = jax.tree.map(lambda x: x + 1, zeros_tree)
print(ones_tree)

Great! We can now create custom layers and we can train even bigger neural networks!

But what if our neural network is really big? How do we handle training it across multiple devices?

I'm glad you asked.

## Parallelism in JAX 💻💻
Parallelism in JAX is handled by another fundamental transform function: pmap

`pmap` basics

In [ ]:
import jax

jax.devices()  # Output in my case for one device:( -> [TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0)]

In [ ]:
x = np.arange(5)  # signal
w = np.array([2., 3., 4.])  # window/kernel

def convolve(w, x):  # implementation of 1D convolution/correlation
    output = []

    for i in range(1, len(x) - 1):
        output.append(jnp.dot(x[i - 1:i + 2], w))

    return jnp.array(output)

result = convolve(w, x)
print(repr(result))

In [ ]:
n_devices = jax.local_device_count()
print(f'Number of available devices: {n_devices}')

# Let's now imagine we have a much heavier load (a batch of examples)
xs = np.arange(5 * n_devices).reshape(-1, 5)
ws = np.stack([w] * n_devices)

print(xs.shape, ws.shape)

In [ ]:
# First way to optimize this is to simply use vmap
vmap_result = jax.vmap(convolve)(ws, xs)
print(repr(vmap_result))

In [ ]:
# The amazing thing is if you just swap vmap for pmap you are now running on multiple
# devices. How cool is that?
pmap_result = jax.pmap(convolve)(ws, xs)
print(repr(pmap_result))  # ShardedDeviceArray!

In [ ]:
# No cross-device communication costs. Computations are done independently on each dev.
double_pmap_result = jax.pmap(convolve)(jax.pmap(convolve)(ws, xs), xs)
print(repr(double_pmap_result))

In [ ]:
# Same results but we don't have to manually broadcast w (recall: same as for vmap!)
pmap_smarter_result = jax.pmap(convolve, in_axes=(None, 0))(w, xs)
print(repr(pmap_smarter_result))

This is all great, but oftentimes we do need to communicate between devices. 📱

Let's see how that is handled.

### Communication between devices 📱📱📱📱

In [ ]:
# Same example as above but this time we communicate across devices
# in order to normalize the outputs

def normalized_convolution(w, x):
    output = []

    for i in range(1, len(x) - 1):
        output.append(jnp.dot(x[i - 1:i + 2], w))

    output = jnp.array(output)  # same result as before

    return output / jax.lax.psum(output, axis_name="batch_dim")  # this is where communication happens

res_pmap = jax.pmap(normalized_convolution, axis_name="batch_dim", in_axes=(None, 0))(w, xs)
res_vmap = jax.vmap(normalized_convolution, axis_name="batch_dim", in_axes=(None, 0))(w, xs)

print(repr(res_pmap))
print(repr(res_vmap))

print(f'Verify the output is normalized: {sum(res_pmap[:, 0])}') # your output is might be different due to num of devices

Great - before we train yet another ML model (this time on multiple devices) let me show you a couple more useful functions!

In [ ]:
# Sometimes aside from grads we also need to return the loss value (for logging, etc.)

def sum_squared_error(x, y):
    return sum((x - y)**2)


x = jnp.arange(4, dtype=jnp.float32)
y = x + 0.1

# An efficient way to return both grads and loss value
jax.value_and_grad(sum_squared_error)(x, y)

In [ ]:
# And sometimes the loss function needs to return intermediate results

def sum_squared_error_with_aux(x, y):
    return sum((x - y)**2), x - y


jax.grad(sum_squared_error_with_aux, has_aux=True)(x, y)  # has_aux=True

How has_aux=True works:
When has_aux=True is provided to jax.grad:

1. JAX executes the function f(x, y).
2. It uses the first element of the output tuple (the loss value) to compute the gradient.
3. It discards the auxiliary data during the backpropagation step (as it's not used in the gradient computation).
4. It returns a tuple: (gradient, auxiliary_data).

In [ ]:
gradient, aux_diff = jax.grad(sum_squared_error_with_aux, has_aux=True)(x, y)

print("Auxiliary Differences:", aux_diff)
print("Gradient:", gradient)

###  Combining value_and_grad with Auxiliary Output

In [ ]:
# Note: value_and_grad returns (value, grad)

value_and_grad_fn = jax.value_and_grad(sum_squared_error_with_aux, has_aux=True)

# The output format is: ((loss_value, aux_data), gradient)
(loss, aux_diff), gradient = value_and_grad_fn(x, y)

print("Loss:", loss)
print("Auxiliary Differences:", aux_diff)
print("Gradient:", gradient)

We're now ready to see the most complex model pipeline so far in JAX.

## Training a very simple model in parallel!

In [ ]:
class Params(NamedTuple):
    weight: jnp.array
    bias: jnp.array

lr = 0.005

def init_model(rng):
    weights_key, bias_key = jax.random.split(rng)
    weight = jax.random.normal(weights_key, ())
    bias = jax.random.normal(bias_key, ())
    return Params(weight, bias)

def forward(params, xs):
    return params.weight * xs + params.bias

def loss_fn(params, xs, ys):
    pred = forward(params, xs)
    return jnp.mean((pred - ys)**2)  # MSE

@functools.partial(jax.vmap, axis_name="batch")
def update(params, xs, ys):
    # Compute the gradients on the given minibatch (individually on each device).
    loss, grads = jax.value_and_grad(loss_fn)(params, xs, ys)

    # Combine the gradient across all devices (by taking their mean).
    grads = jax.lax.pmean(grads, axis_name="batch")

    # Also combine the loss. Unnecessary for the update, but useful for logging.
    loss = jax.lax.pmean(loss, axis_name="batch")

    # Each device performs its own SGD update, but since we start with the same params
    # and synchronise gradients, the params stay in sync on each device
    new_params = jax.tree.map(
        lambda param, g: param - g * lr, params, grads
    )

    # If we were using Adam or another stateful optimizer,
    # we would also do something like:
    # updates, new_optimizer_state = optimizer(grads, optimizer_state)
    # and then use updates instead of grads to actually update the params.
    # (And we'd include the new_optimizer_state in the output, naturally.)

    return new_params, loss

In [ ]:
true_w, true_b = 2, -1
xs = np.random.normal(size=(128, 1))
noise = 0.5 * np.random.normal(size=(128, 1))
ys = xs * true_w + true_b + noise

plt.scatter(xs, ys)
plt.show()

In [ ]:
state = 0
params = init_model(jax.random.PRNGKey(state))
n_devices = jax.local_device_count()
replicated_params = jax.tree.map(lambda x: jnp.array([x] * n_devices), params)
print(replicated_params)

def reshape_for_pmap(data, n_device):
    return data.reshape(n_devices, data.shape[0] // n_devices, *data.shape[1:])

x_parallel = reshape_for_pmap(xs, n_devices)
y_parallel = reshape_for_pmap(ys, n_devices)

print(x_parallel.shape, y_parallel.shape)

In [ ]:
def type_after_update(name, obj):
    print(f"After the first 'update()', '{name}' is a {type(obj)}")

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    # This is where the params and data gets communicated to devices
    replicated_params, loss = update(replicated_params, x_parallel, y_parallel)

    # replicated_params and loss are now both ShardedDeviceArrays,
    # indicating that they're on the devices.
    # x/y_parallel remains a NumPy array on the host (simulating data streaming).
    if epoch == 0:
        type_after_update('replicated_params.weight', replicated_params.weight)
        type_after_update('loss', loss)
        type_after_update('x_parallel', x_parallel)

    if epoch % 100 == 0:
        print(loss.shape)
        print(f"Step {epoch:3d}, loss: {loss[0]:.3f}")

# Like the loss, the leaves of params have an extra leading dimension,
# so we take the params from the first device.
params = jax.device_get(jax.tree.map(lambda x: x[0], replicated_params))

In [ ]:
plt.scatter(xs, ys, c='g', marker='.', label='GT')
plt.plot(xs, forward(params, xs), c='r', label="Prediction")
plt.legend()
plt.show()

Wohoo! We trained a model in parallel on multiple devices (in my case one, but should work for multiple devices)! 🥳🥳🥳

There are more super important tools we're missing though.

1. How would we go about transfer learning in JAX?
2. How do we freeze certain layers and fine tune others?
3. How do we get per sample gradients (and not the usual per batch gradients)?
And so on. That brings us to the last section!

### Advanced Autodiff 🚀

### Stop gradients
`jax.lax.stop_gradient` is the primitive used for this purpose.
**Example 1: TD(0) update (RL algorithm).**
The TD(0) update to the network parameters is:
$$\Delta \theta = (r_t + v_{\theta}(s_t) - v_{\theta}(s_{t-1})) \nabla v_{\theta}(s_{t-1})
$$
This update is not the gradient of any loss function.
However, it can be **written** as the gradient of the pseudo loss function

$$L(\theta) = [r_t + v_{\theta}(s_t) - v_{\theta}(s_{t-1})]^2$$
if the dependency of the target $r_t + v_{\theta}(s_t)$ on the parameter $\theta$ is ignored."

In [ ]:
# value function (simple linear fn) and initial parameters
value_fn = lambda theta, state: jnp.dot(theta, state)
theta = jnp.array([0.1, -0.1, 0.])

# an example transition
s_tm1 = jnp.array([1., 2., -1.])
r_t = jnp.array(1.)
s_t = jnp.array([2., 1., 0.])

def td_loss(theta, s_tm1, r_t, s_t):
    v_tm1 = value_fn(theta, s_tm1)
    target = r_t + value_fn(theta, s_t)
    return (jax.lax.stop_gradient(target) - v_tm1)**2

td_update = jax.grad(td_loss)
delta_theta = td_update(theta, s_tm1, r_t, s_t)

delta_theta

**Example 2: straight-through estimator**

(used in e.g. VQ-VAE, check out my video: https://www.youtube.com/watch?v=VZFVUrYcig0)

In [ ]:
def f(x):
    return jnp.round(x)  # non-differentiable

def straight_through_f(x):
    return x + jax.lax.stop_gradient((f(x) - x))  # f(x) = x => dfdx = 1

x = 5.6
print(f"f(x) = {f(x)}")
print("straight_through_f(x):", straight_through_f(x))  # same values in the forward pass

print("grad(f)(x):", jax.grad(f)(x))  # non-diff so it just returns 0
print("grad(straight_through_f)(x):", jax.grad(straight_through_f)(x))

### Per sample gradients

"In many frameworks (PyTorch, TF, Theano) it is often not trivial to compute per-example gradients, because the library directly accumulates the gradient over the batch. Naive workarounds, such as computing a separate loss per example and then aggregating the resulting gradients are typically very inefficient."

In [ ]:
# Here is how to do it in JAX!

# Batch the data

batched_s_tm1 = jnp.stack([s_tm1, s_tm1])
batched_r_t = jnp.stack([r_t, r_t])
batched_s_t = jnp.stack([s_t, s_t])

perex_grads = jax.jit(jax.vmap(jax.grad(td_loss), in_axes=(None, 0, 0, 0)))
perex_grads(theta, batched_s_tm1, batched_r_t, batched_s_t)

### Advanced stuff 🐉
JAX autodiff engine is very powerful.

You can do various things like define custom derivative rules, etc. Take a look at advanced tutorials in the docs (e.g. [JAX: Autodiff Cookbook](https://docs.jax.dev/en/latest/notebooks/autodiff_cookbook.html))

Here is an example of how to calculate update for MAML (notice how similar the code is to math!)

![](https://github.com/Xrenya/quickstart_JAX/blob/main/images/meta-learning.png?raw=true)

![](https://github.com/Xrenya/quickstart_JAX/blob/main/images/meta-curve.png?raw=true)

```python
# Very easy to do in JAX
def meta_loss_fn(params, data):
    """Computes the loss after one step of SGD."""
    grads = jax.grad(loss_fn)(params, data)
    return loss_fn(params - lr * grads, data)

meta_grads = jax.grad(meta_loss_fn)(params, data)
```